In [1]:
import os

In [2]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\stock\\Power-Time-Series-App\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\stock\\Power-Time-Series-App'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    training_data: Path

In [6]:
from powerTimeSeries.constants import *
from powerTimeSeries.utils.common import read_yaml, create_directories
import pandas as pd


In [7]:
import pandas as pd
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_training_config(self) -> TrainingConfig:
        config = self.config.training

        create_directories([
                Path(config.root_dir)
            ])

        training_config = TrainingConfig(
            root_dir=Path(config.root_dir),
            trained_model_path=Path(config.trained_model_path),
            training_data=Path(config.training_data),
        )

        return training_config


In [8]:
import time

In [9]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import xgboost as xgb
import json

class Training:
    def __init__(self, config:TrainingConfig):
        self.config = config

    def training(self):
        df = pd.read_csv(self.config.training_data)

        self.train_model(df)

    def train_model(self, df):
        FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month', 'year',
                    'lag1', 'lag2', 'lag3']
        TARGET = 'PJME_MW'

        X_all = df[FEATURES]
        y_all = df[TARGET]

        print("Training process ongoing")

        reg = XGBRegressor(base_score=0.5,
                           booster='gbtree',
                           n_estimators=500,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.01)
        reg.fit(X_all, y_all,
                eval_set=[(X_all, y_all)],
                verbose=100)
        

        self.save_model(reg)

    def save_model(self, model):
        model.save_model(self.config.trained_model_path)
        print("Model saved successfully.")


In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.training()
except Exception as e:
    raise e

[2023-07-23 19:29:10,391: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-23 19:29:10,396: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-23 19:29:10,399: INFO: common: created directory at: artifacts]
[2023-07-23 19:29:10,403: INFO: common: created directory at: artifacts\training]
Training process ongoing
[19:29:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:32402.65859
[100]	validation_0-rmse:12426.27499
[200]	validation_0-rmse:5752.77330
[300]	validation_0-rmse:3970.98141
[400]	validation_0-rmse:3571.13391
[499]	validation_0-rmse:3455.70822
Model saved successfully.
